**IMPORTING REQUIRED LIBRARIES**

In [0]:
import numpy as np
from keras.datasets import cifar10
import cv2
import keras as k
from keras import models
from matplotlib import pyplot as p
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model
from keras import optimizers
from keras.layers import Dense, Dropout, Activation, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.applications.xception import Xception
from keras.applications import VGG16
import pandas as pd
import os 
from PIL import Image
from sklearn.metrics import accuracy_score
import autokeras as ak
from keras.utils import np_utils
from ann_visualizer.visualize import ann_viz;

**LOADING AND PREPROCESSING IMAGE AS PER THE REQUIRED FORMAT**

In [0]:

def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')

    X_train=np.zeros([num_classes*500,32,32,3],dtype='uint8')
    y_train=np.zeros([num_classes*500], dtype='uint8')

    trainPath=path+'/train/train/bilinear'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            X=np.array(Image.open(os.path.join(sChildPath,c)))
            if len(np.shape(X))==2:
                X_train[i]=np.array([X,X,X])
            else:
                X_train[i]=X
            y_train[i]=j
            i+=1
        j+=1
        if (j >= num_classes):
            break

    print('finished loading training images')

    val_annotations_map = get_annotations_map()

    X_test = np.zeros([num_classes*50,32,32,3],dtype='uint8')
    y_test = np.zeros([num_classes*50], dtype='uint8')


    print('loading test images...')

    i = 0
    testPath=path+'/val/val/bilinear/images'
    for sChild in os.listdir(testPath):
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(testPath, sChild)
            X=np.array(Image.open(sChildPath))
            if len(np.shape(X))==2:
                X_test[i]=np.array([X,X,X])
            else:
                X_test[i]=X
            y_test[i]=annotations[val_annotations_map[sChild]]
            i+=1
        else:
            pass


    print('finished loading test images'+str(i))

    return X_train,y_train,X_test,y_test

**MAPPING TEST IMAGES WITH THEIR ANNOTATIONS FOR LABEL INFORMATION**

In [0]:
def get_annotations_map():
	valAnnotationsPath = 'C:/Users/Abhishek/data/val/val/bilinear/val_annotations.txt'
	valAnnotationsFile = open(valAnnotationsPath, 'r')
	valAnnotationsContents = valAnnotationsFile.read()
	valAnnotations = {}

	for line in valAnnotationsContents.splitlines():
		pieces = line.strip().split()
		valAnnotations[pieces[0]] = pieces[1]

	return valAnnotations

In [0]:
path='C:/Users/Abhishek/data'
x_train,y_train,x_test,y_test=load_images(path,200)

Loading 200 classes
loading training images...
finished loading training images
loading test images...
finished loading test images10000


**PREPROCESSING DATA**

In [0]:
print('X_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

num_samples=len(x_train)

# input image dimensions
num_channels , img_rows, img_cols = x_train.shape[1], x_train.shape[2], x_train.shape[3]

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, 200)
Y_test = np_utils.to_categorical(y_test, 200)

    

X_train shape: (100000, 32, 32, 3)
100000 train samples
10000 test samples


**TRAINING MODEL WITH AUTOKERAS**



*   Time limit chosen for best model: 10 hours




In [0]:
clf = ak.ImageClassifier(verbose=True, augment=True)
clf.fit(x_train, y_train, time_limit=(10*60*60))

clf.final_fit(x_train, y_train, x_test, y_test, retrain=False)


Saving Directory: C:\Users\Abhishek\AppData\Local\Temp\autokeras_CEANP1
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+

Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   14.692697238922118   |         0.1856         |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+

Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss 

NameError: name 'accuracy_score' is not defined

**DIFFERENT WAYS OF EXPORTING THE BEST MODEL OBTAINED**

In [0]:
model_final = clf.export_autokeras_model

In [0]:
model_final

<bound method ImageClassifier.export_autokeras_model of <autokeras.image.image_supervised.ImageClassifier object at 0x000001CEBD102CF8>>

In [0]:
model_keras = clf.export_keras_model

In [0]:
model_keras

<bound method DeepTaskSupervised.export_keras_model of <autokeras.image.image_supervised.ImageClassifier object at 0x000001CEBD102CF8>>


In [0]:
# save model and weights 
save_dir = os.path.join(os.getcwd(),'saved_models')
model_name = 'autokeras_imgnet200_new.h5'


**MODEL TEST ACCURACY**

In [0]:
pred = clf.evaluate(x_test,y_test)
print(pred)
# Manual predict
y_prediction = clf.predict(x_test)
accuracy_score(y_true=y_test, y_pred=y_prediction)

0.4681


0.4681

**SOME EXTRA SAMPLES OF AUTOKERAS MODELS RUN WITH DIFFERENT TIME LIMITS**

In [0]:
clf_new = ak.ImageClassifier(verbose=True, augment=True)
# Train for 9 hours
clf_new.fit(x_train, y_train, time_limit=(9*60*60))
clf_new.final_fit(x_train, y_train, x_test, y_test, retrain=False)



Saving Directory: C:\Users\Abhishek\AppData\Local\Temp\autokeras_KC9PDT
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+

Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   15.420692348480225   |  0.16440000000000002   |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+

Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss 

In [0]:
clf_new.export_autokeras_model('C:\\Users\\Abhishek\\autokeras_models')

In [0]:
import os
from graphviz import Digraph

from autokeras.utils import pickle_from_file


def to_pdf(graph, path):
    dot = Digraph(comment='The Round Table')

    for index, node in enumerate(graph.node_list):
        dot.node(str(index), str(node.shape))

    for u in range(graph.n_nodes):
        for v, layer_id in graph.adj_list[u]:
            dot.edge(str(u), str(v), str(graph.layer_list[layer_id]))

    dot.render(path)

Using TensorFlow backend.


In [0]:
def visualize(path):
    cnn_module = pickle_from_file(os.path.join(path, 'module'))
    cnn_module.searcher.path = path
    for item in cnn_module.searcher.history:
        model_id = item['model_id']
        graph = cnn_module.searcher.load_model_by_id(model_id)
        to_pdf(graph, os.path.join(path, str(model_id)))


In [0]:
clf_final= ak.ImageClassifier(path='C:\\Users\\Abhishek\\AutoKeras_visualization',verbose=True, augment=True)
# Train for 15 hours
clf_final.fit(x_train, y_train, time_limit=(15*60*60))

clf_final.final_fit(x_train, y_train, x_test, y_test, retrain=False)



Saving Directory: C:\Users\Abhishek\AutoKeras_visualization
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+

Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   15.470404291152954   |  0.16560000000000002   |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+

Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |  

**FINAL CONCLUSIONS**


*   Model that ran for 10 hours seems to give the maximum accuracy.
*   It is not a feasible solution to run a model for 24 hours inorder to get an optimal model. 
*   When compared with task 1 of fine tuning model manually, autokeras seems to perform better and also saves time from trial and error.



